# HW04 : Problem 2


## Part O

In [ ]:
from pprint import pprint
from visual_dataset import getGroundTruthList
pprint(getGroundTruthList(17))

In [ ]:
from visual_dataset import visual_dataset
visual_dataset(17, show_name=False, fontsize=20, figsize=(10,10))
visual_dataset(17)

## Part I

In [ ]:
from prepare_data import prepare_data
import matplotlib.pyplot as plt


number = 17
cropped_images, classes = prepare_data(number)

fig = plt.figure(figsize=(15,8))
fig.subplots_adjust(hspace=0.5)
for i in range(20):
  plt.subplot(4,5,i+1)
  plt.imshow(cropped_images[i])
  plt.grid(False); plt.xticks([]); plt.yticks([])
  plt.title(classes[i].name, fontdict={'color':'black', 'fontsize':16})



In [ ]:
categories = [cat.value for cat in classes]
categories 

## Part II

In [ ]:
from transform import resize_image

img = cropped_images[0]
plt.imshow(resize_image(img, (224,224)))
plt.show()

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from prepare_data import getGroundTruthList, ObjectType
from prepare_data import get_number_of_files


number_of_labeled_files, number_of_nonlabeled_files = get_number_of_files()

all_classes = np.array([]);
for i in range(1,number_of_labeled_files):
  groundTruthList = getGroundTruthList(i)
  classes_i = np.array([obj.object_type.value for obj in groundTruthList])
  all_classes = np.concatenate((all_classes, classes_i), axis=0)
all_classes = np.concatenate((all_classes, np.zeros(number_of_nonlabeled_files)), axis=0)

plt.figure(figsize=(20,5))
ax = plt.subplot(111)
ax.hist(all_classes, bins=np.arange(0,12), rwidth=0.75, edgecolor='black')
ax.set_xticks(.5+np.arange(11))
ax.set_xlim(0,11)
ax.set_xticklabels([ObjectType(i).name for i in range(11)])
plt.show()


counts, bins = np.histogram(all_classes,bins=np.arange(-0.5,11.5), density=False)
P_removal = (counts - counts.min()) / counts

np.savez_compressed('unbalanced-info.npz', counts=counts, bins=bins, P_removal=P_removal, num_positive=number_of_labeled_files, num_negative=number_of_nonlabeled_files)

> make it balanced

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm 
from prepare_data import getGroundTruthList, ObjectType
from prepare_data import load_negative, load_positive, prepare_data

np.random.seed(0)
target_size = (224, 224)

all_classes = np.array([],dtype=np.uint8);
all_images = np.zeros((0,*target_size,3));
for i in tqdm(range(1,number_of_labeled_files), desc="Loading…",  ascii=False, ncols=75): 
    images_i, classes_i = prepare_data(i)
    classes_i = np.array([cat.value for cat in classes_i])
    index_to_remove = np.where(P_removal[classes_i] > np.random.uniform(size=classes_i.shape))[0]
    len_i = classes_i.shape[0];
    classes_i = np.array([classes_i[k] for k in range(len_i) if k not in index_to_remove])
    if classes_i.shape[0] == 0: continue
    images_i = np.array([resize_image(images_i[k],target_size) for k in range(len_i) if k not in index_to_remove])
     
    all_images = np.concatenate((all_images, images_i), axis=0).astype(np.uint8)
    all_classes = np.concatenate((all_classes, classes_i), axis=0)

classes_i = np.zeros(number_of_nonlabeled_files, dtype=np.int64)
images_i = [load_negative(i+1) for i in range(number_of_nonlabeled_files)]
len_i = classes_i.shape[0]
index_to_remove = np.where(P_removal[classes_i] > np.random.uniform(size=classes_i.shape))[0]
classes_i = np.array([classes_i[k] for k in range(len_i) if k not in index_to_remove])
if classes_i.shape[0] > 0:
  images_i = np.array([resize_image(images_i[k],target_size) for k in range(len_i) if k not in index_to_remove])
  all_classes = np.concatenate((all_classes,classes_i), axis=0)
  all_images = np.concatenate((all_images, images_i), axis=0).astype(np.uint8)

np.savez_compressed('all_images.npz', all_images)
np.savez_compressed('all_classes.npz', all_classes)

plt.figure(figsize=(20,5))
ax = plt.subplot(111)
ax.hist(all_classes, bins=np.arange(0,12), rwidth=0.75, edgecolor='black')
ax.set_xticks(.5+np.arange(11))
ax.set_xlim(0,11)
ax.set_xticklabels([ObjectType(i).name for i in range(11)])
plt.show()


## Part III

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

all_categories = to_categorical(all_classes)
all_image_data = all_images.astype(np.float)/255.
x_train, x_test, y_train, y_test = train_test_split(all_image_data, all_categories, test_size = 0.2, shuffle=True) 
x_subtrain, x_valid, y_subtrain, y_valid = train_test_split(x_train, y_train, test_size = 0.2, shuffle=True) 

In [ ]:
# from tesorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Dense
# load the model
pre_model = DenseNet121(weights='imagenet')
# Freeze all the layers
for layer in pre_model.layers[:]:
    layer.trainable = False

output = Dense(11,  activation='sigmoid', name='output')(pre_model.layers[-2].output)
pre_model = Model(inputs=pre_model.inputs, outputs=output)
pre_model.summary()
# plot_model(pre_model, show_shapes=True)


In [ ]:
pre_model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=['accuracy'])
history = pre_model.fit(x_subtrain,
                    y_subtrain,
                    epochs=20,
                    verbose=2,
                    validation_data=(x_valid, y_valid),
                    batch_size=16
                    )

# save model and architecture to single file
pre_model.save("pre_train.h5")
print("Saved model to disk")

In [ ]:
score, acc = pre_model.evaluate(x_test, y_test, batch_size=16)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
yhat = pre_model.predict(x_test, batch_size=16)
yhat = yhat.argmax(axis=1) 
acc = np.sum(yhat==y_test.argmax(axis=1))/yhat.shape[0]
print('Real Test Accuracy:', acc)

## Part IV

### To test how it works

> Create sample dataset to see what is happend in DataAugmentaion process 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import requests
from transform import resize_image

url = 'https://github.com/dufourpascal/stepupai/raw/master/tutorials/data_augmentation/image.jpg'
r = requests.get(url, allow_redirects=True)
open('image.jpg', 'wb').write(r.content)
url = 'https://github.com/dufourpascal/stepupai/raw/master/tutorials/data_augmentation/image_town.jpg'
r = requests.get(url, allow_redirects=True)
open('image_town.jpg', 'wb').write(r.content)

size=(800,450)

image = load_img('image.jpg')
image = img_to_array(image).astype(np.uint8)
data1 = np.expand_dims(resize_image(image,size), 0).astype(int)
plt.axis('off')
plt.imshow(data1[0])
plt.show()

image_town = load_img('image_town.jpg')
image_town = img_to_array(image_town).astype(np.uint8)
data2 = np.expand_dims(resize_image(image_town,size), 0).astype(int)
plt.axis('off')
plt.imshow(data2[0])
plt.show()
data=np.concatenate((data1,data2),axis=0)

> without changes:

In [ ]:
from transform import DataAugmentation
datagen = DataAugmentation(data)
datagen.plot(n_images=5)

> Width and Height Shift:

In [ ]:
datagen = DataAugmentation(data)
datagen.width_shift_range = 0.2
datagen.height_shift_range = 0.2
datagen.plot(n_images=5)

> Image Flips:

In [ ]:
datagen = DataAugmentation(data)
datagen.horizontal_flip = True
datagen.vertical_flip = True
datagen.plot(n_images=5)

> Rotation:

In [ ]:
datagen = DataAugmentation(data)
datagen.shear_range = 20
datagen.plot(n_images=5)

> Zoom:

In [ ]:
datagen = DataAugmentation(data)
datagen.zoom_range = [0.5, 1.5]
datagen.plot(n_images=5)

> Shear:

In [ ]:
datagen = DataAugmentation(data)
datagen.shear_range = 20
datagen.plot(n_images=5)

> Brightness:

In [ ]:
datagen = DataAugmentation(data)
datagen.brightness_range = [0.5, 2.0]
datagen.plot(n_images=5)

> Combining Multiple Transformations for Data Augmentation:

In [ ]:
datagen = DataAugmentation(data,
              progressbar = True,
              fill_mode='nearest',
							horizontal_flip=True,
							width_shift_range=0.2,
							height_shift_range=0.2,
							zoom_range=[0.8, 1.2],
							rotation_range=20,
							shear_range=10,
							brightness_range = [0.75, 1.5])
datagen.plot(n_images=5)

### Use it and make dataset balanced

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

from prepare_data import getGroundTruthList, ObjectType
from prepare_data import prepare_data, load_negative
from transform import resize_image

info = np.load('unbalanced-info.npz') #list(info.keys())
counts, bins, P_removal, num_positive, num_negative = info['counts'], info['bins'], info['P_removal'], info['num_positive'], info['num_negative']

np.random.seed(0)
target_size = (224, 224)

all_classes = np.array([],dtype=np.uint8);
all_images = np.zeros((0,*target_size,3));
for i in tqdm(range(1, num_positive), desc="Loading…",  ascii=False, ncols=75): 
    images_i, classes_i = prepare_data(i)
    classes_i = np.array([cat.value for cat in classes_i]);    len_i = classes_i.shape[0];
    images_i = np.array([resize_image(images_i[k],target_size) for k in range(len_i)])
     
    all_images = np.concatenate((all_images, images_i), axis=0).astype(np.uint8)
    all_classes = np.concatenate((all_classes, classes_i), axis=0)

classes_i = np.zeros(num_negative, dtype=np.int64)
images_i = [load_negative(i+1) for i in range(num_negative)];      len_i = classes_i.shape[0]
images_i = np.array([resize_image(images_i[k],target_size) for k in range(len_i)])
all_classes = np.concatenate((all_classes,classes_i), axis=0)
all_images = np.concatenate((all_images, images_i), axis=0).astype(np.uint8)


num_images = all_classes.shape[0]
all_data = [[] for _ in range(11)]
for i in tqdm(range(1, num_images), desc="Preparing…",  ascii=False, ncols=75): 
    all_data[all_classes[i]].append(all_images[i])
all_data = [np.array(i) for i in all_data]
np.savez_compressed('all_data.npz', all_data)

In [ ]:
data = all_data[0]
print(data.shape)
datagen = DataAugmentation(data[:30])
images = datagen.get(n_images=5)
print(images.shape)

In [ ]:
import shutil, os

try:
  augmented_folder = 'augmented'
  augmented_subfolder = os.path.join(augmented_folder,'{}')
  shutil.rmtree(augmented_folder)
  os.mkdir(augmented_folder)
  for i in range(11):
    print(augmented_subfolder.format(i))
    os.mkdir(augmented_subfolder.format(i))
except:
    pass
    
i = 0
for batch in datagen.flow(data, batch_size=1, save_to_dir=augmented_subfolder.format(0), save_prefix='', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely




In [ ]:
len([name for name in os.listdir(augmented_subfolder.format(0))])

# Part V

## Part VI

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from skimage.transform import pyramid_gaussian
from transform import resize_image

from prepare_data import get_number_of_files, load_negative, load_positive, prepare_data, getGroundTruthList
num_scales = 4
image = load_positive(17)
print(image.shape)
target_size = (224,224)
scale= min(target_size)/min(image.shape[:-1]) *1.25* 2**(num_scales-1)
image = resize_image(image, target_size= (int(image.shape[1]*scale),int(image.shape[0]*scale)))
print(image.shape)

pyramid = tuple(pyramid_gaussian(image, max_layer=num_scales-1, downscale=2, multichannel=True))

fig, ax = plt.subplots(1, num_scales, gridspec_kw={'width_ratios': [2**i for i in range(num_scales-1,-1,-1)]})
for i in range(num_scales):
    ax[i].imshow(pyramid[i])
fig.set_size_inches(20,5)

In [ ]:
overlap = 0.2
def get_image_slides(image,overlap=0.2,target_size=target_size):
    def calculate_number(tar_size,im_size,overlap):
      return int((im_size/tar_size-1)/(1-overlap))
    ncol = calculate_number(target_size[1],image.shape[1],overlap)
    nrow = calculate_number(target_size[0],image.shape[0],overlap)
    images = []
    for col in range(ncol):
      for row in range(nrow):
        c_start = col*int(target_size[1]*(1-overlap))
        c_end = c_start + target_size[1]    
        r_start = row*int(target_size[0]*(1-overlap))
        r_end = r_start + target_size[0]
        # print(r_start,r_end,c_start,c_end)
        img = image[r_start:r_end,c_start:c_end,:]
        images.append(img)
    images = np.array(images)
    return images

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('pre_train.h5')

In [ ]:
images = get_image_slides(pyramid[0],overlap=0.75,target_size=target_size).astype(np.float) / 255.
# images = x_test;
yhat = model.predict(images)
######
# def estimate(images, thresh=0.2):
#   yhat > thresh
# index = []
# for i in range(yhat.shape[0]):
#   ind = np.where(yhat[i]>0.)[0]
#   ind_max = yhat[i][ind].argmax()
#   print(ind_max)
#   ind_max = ind_max if ind_max.shape else 0
#   index.append(ind_max)